# ICE-NMF Parameter Tunning palco_2010

In [1]:
#imports
import numpy as np 
import csv
import re
import sys
import math
import matplotlib.pyplot as plt
from numpy import linalg as L
import pandas as pd
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from data import ImplicitData
import pandas as pd
from BISGD import BISGD
from ICE_NMF import ICE_NMF
from collections import defaultdict
from eval_implicit import EvalPrequential
from datetime import datetime
import getopt

## Functions


In [2]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
  
    #new = normalize(V[:,np.newaxis], axis=0).ravel()
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [3]:
sys.path.append('./')
configPath = 'config'
dataPath = 'palco_2010.tsv'


Data = []
data = pd.read_csv("palco_2010.tsv","\t")
data = data.head(n=58800)
print(data)
Data = data.values.tolist()



/home/mmelo/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


       user_id  track_id            timestamp
0         4220     31313  2010-01-01 00:10:36
1         4220     29567  2010-01-01 00:13:07
2         4220     31313  2010-01-01 00:13:18
3         3970     34925  2010-01-01 00:39:21
4         6258     18106  2010-01-01 00:46:25
...        ...       ...                  ...
58795    47350      9040  2010-01-24 11:04:20
58796    47350      9034  2010-01-24 11:04:32
58797    47350      9032  2010-01-24 11:05:09
58798    47350      9030  2010-01-24 11:05:11
58799    47350      9031  2010-01-24 11:05:17

[58800 rows x 3 columns]


# Parameter Study

## Iterations

In [4]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

iterations = [5,10,15,20,25,30]

for i in iterations:
    print("iter")
    print(i)
    num_clusters = 8
    cl_num_iterations = i
    cl_learn_rate = 0.1
    cl_regularization = 0.1
    num_factors = 160
    num_iter = 4
    learn_rate = 0.5
    regularization = 0.4
    model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update']))    


iter
5
start time 2022-06-27 22:52:25.932615


/home/mmelo/NMF|BISGD/data/implicit_data.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if user not in self.userset:
/home/mmelo/NMF|BISGD/data/implicit_data.py:62: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if item not in self.itemset:


npmean(resuls[Recall@20]) 0.20648521505376344
end time 2022-06-27 23:15:04.958642
run time 0:22:39.026027

get tuple 7.173848800918683e-06
recommend 0.21279635308410533
eval_point 3.1933869405459335e-05
update 0.001555542856657586
iter
10
start time 2022-06-27 23:15:05.094078
npmean(resuls[Recall@20]) 0.2122753475754493
end time 2022-06-27 23:38:14.899564
run time 0:23:09.805486

get tuple 7.451059055977127e-06
recommend 0.21787065132143701
eval_point 3.238005652675066e-05
update 0.0017648179798710102
iter
15
start time 2022-06-27 23:38:15.021250
npmean(resuls[Recall@20]) 0.21140380126708902
end time 2022-06-28 00:01:56.800227
run time 0:23:41.778977

get tuple 7.207519343110169e-06
recommend 0.21794427832431418
eval_point 3.221731418051216e-05
update 0.0019307842789864054
iter
20
start time 2022-06-28 00:01:56.919212
npmean(resuls[Recall@20]) 0.20914246434095204
end time 2022-06-28 00:26:21.614267
run time 0:24:24.695055

get tuple 7.947890125975317e-06
recommend 0.2297127563163861
ev

## Eta

In [5]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])


etas = [0.0001, 0.001, 0.01, 0.1]

for e in etas:
    print("eta")
    print(e)
    num_clusters = 8
    cl_num_iterations = 25
    cl_learn_rate = e
    cl_regularization = 0.1
    num_factors = 160
    num_iter = 4
    learn_rate = 0.5
    regularization = 0.4
    model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

eta
0.0001
start time 2022-06-28 01:13:43.208564
npmean(resuls[Recall@20]) 0.20957482423836626
end time 2022-06-28 01:37:13.401709
run time 0:23:30.193145

get tuple 7.428364688847341e-06
recommend 0.21425547131256473
eval_point 3.2826933846411434e-05
update 0.0021973208180901143
eta
0.001
start time 2022-06-28 01:37:13.542510
npmean(resuls[Recall@20]) 0.21397830951971078
end time 2022-06-28 02:01:02.553167
run time 0:23:49.010657

get tuple 7.744873462080144e-06
recommend 0.22227397997667836
eval_point 3.379744452399176e-05
update 0.002325842781942718
eta
0.01
start time 2022-06-28 02:01:02.690429
npmean(resuls[Recall@20]) 0.22003034901365706
end time 2022-06-28 02:25:40.354471
run time 0:24:37.664042

get tuple 7.930361494726064e-06
recommend 0.22578461209189288
eval_point 3.336835040187498e-05
update 0.002337498198561117
eta
0.1
start time 2022-06-28 02:25:40.483698
npmean(resuls[Recall@20]) 0.2058874160784989
end time 2022-06-28 02:49:17.703635
run time 0:23:37.219937

get tuple 7.

## Lambda

In [6]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])

lambdas = [0,0.0001, 0.001, 0.01, 0.1]   

for l in lambdas:
    print("lambda")
    print(l)
    num_clusters = 8
    cl_num_iterations = 25
    cl_learn_rate = 0.001
    cl_regularization = l
    num_factors = 160
    num_iter = 4
    learn_rate = 0.5
    regularization = 0.4
    model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 10)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 10)

    print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('')
    print('get tuple',np.mean(results['time_get_tuple']))
    print('recommend',np.mean(results['time_recommend']))
    print('eval_point',np.mean(results['time_eval_point']))
    print('update',np.mean(results['time_update'])) 

lambda
0
start time 2022-06-28 02:49:18.219285
npmean(resuls[Recall@20]) 0.20926243567753003
end time 2022-06-28 03:13:26.526700
run time 0:24:08.307415

get tuple 8.078671637035551e-06
recommend 0.22408634467149477
eval_point 3.3380318669471286e-05
update 0.0023943171614692325
lambda
0.0001
start time 2022-06-28 03:13:26.650863
npmean(resuls[Recall@20]) 0.21600274725274726
end time 2022-06-28 03:37:33.302692
run time 0:24:06.651829

get tuple 7.814055397396995e-06
recommend 0.2249065780868897
eval_point 3.439826624734061e-05
update 0.0023082946838975764
lambda
0.001
start time 2022-06-28 03:37:33.431582
npmean(resuls[Recall@20]) 0.20976767848058334
end time 2022-06-28 04:00:57.129230
run time 0:23:23.697648

get tuple 7.496013933298539e-06
recommend 0.2154823048995757
eval_point 3.197271176590967e-05
update 0.0022445950581102955
lambda
0.01
start time 2022-06-28 04:00:57.252712
npmean(resuls[Recall@20]) 0.2159033719174635
end time 2022-06-28 04:24:10.932337
run time 0:23:13.679625

ge